In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import os
from tensorflow.python.client import device_lib
from collections import Counter

In [2]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

f = open("train.pickle", 'rb')
full_data = pickle.load(f)
f.close()

In [3]:
# Model Description
sense_word = 'hard'
model_name = 'basic'
model_dir = 'output/' + sense_word + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [4]:
word_counts = Counter(full_data[sense_word][1])
print(word_counts)
total_count = len(full_data[sense_word][1])
weights = np.multiply(10, [1 - count/total_count for word, count in word_counts.items()])
weights = weights.astype(np.float32)
print(weights)

Counter({'HARD1': 3455, 'HARD2': 502, 'HARD3': 376})
[ 8.84144974  2.02630973  9.13224125]


In [5]:
# Parameters
mode = 'train'
num_senses = 3
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 5000
decay_rate = 0.96
clip_norm = 1
clipping = True

In [6]:
# MODEL
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

with tf.variable_scope("lstm"):
    cell_fw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=input_keep_prob)
    d_cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=input_keep_prob)
    
    (fw_h, bw_h), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw, d_cell_bw, Wx, sequence_length=x_len, dtype='float', scope='lstm')
    h = tf.concat([fw_h, bw_h], 2)

def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], -0.1, 0.1), name="W_att")
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)
        
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([num_senses]), name="b")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W) + b
    predictions = tf.argmax(logits, 1)

class_weight = tf.constant(weights)
weighted_logits = logits * class_weight
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=weighted_logits, labels=y))

global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [7]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"
print (device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14741389415763459425
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 7958282240
locality {
  bus_id: 1
}
incarnation: 5986465160694741360
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:0a:00.0"
]


In [8]:
# Splitting
data_x = full_data[sense_word][0]
data_y = full_data[sense_word][2]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.8, shuffle=True, stratify=data_y)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, shuffle=True, stratify=y_train)

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
def data_prepare(x):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)

    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            
    return xx, xx_mask

def eval_f1(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    return f1*100

def model(xx, yy, mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {x:xx[s:e], y:yy[s:e], x_mask:mask[s:e], is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [10]:
x_id_train, mask_train = data_prepare(x_train)
x_id_val, mask_val = data_prepare(x_val)

In [11]:
num_epochs = 30

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Val F1 Score: ",  eval_f1(y_val, val_pred), "loss: ", val_loss)

Epoch: 1 Step: 48 loss: 12.7996180256
Val F1 Score:  50.8177208011 loss:  6.67111463547
Epoch: 2 Step: 96 loss: 5.41195377211
Val F1 Score:  48.4996091576 loss:  4.61158595085
Epoch: 3 Step: 144 loss: 3.55104586482
Val F1 Score:  39.4524959742 loss:  2.73180937767
Epoch: 4 Step: 192 loss: 2.75240691255
Val F1 Score:  46.0541719881 loss:  2.29493297338
Epoch: 5 Step: 240 loss: 1.96881641696
Val F1 Score:  45.4455629238 loss:  1.82315752506
Epoch: 6 Step: 288 loss: 1.58194069192
Val F1 Score:  42.3776173929 loss:  1.5729793787
Epoch: 7 Step: 336 loss: 1.27647385622
Val F1 Score:  46.5661242022 loss:  1.35206398964
Epoch: 8 Step: 384 loss: 1.10049246127
Val F1 Score:  41.5799888116 loss:  1.19258269072
Epoch: 9 Step: 432 loss: 0.947192534804
Val F1 Score:  38.323514973 loss:  1.21274006367
Epoch: 10 Step: 480 loss: 0.944974546631
Val F1 Score:  45.4451345756 loss:  0.920793282986
Epoch: 11 Step: 528 loss: 0.862332396209
Val F1 Score:  53.849367317 loss:  0.99055492878
Epoch: 12 Step: 576 

In [12]:
num_epochs = 30

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Val F1 Score: ",  eval_f1(y_val, val_pred), "loss: ", val_loss)

Epoch: 1 Step: 1488 loss: 0.452111756119
Val F1 Score:  38.5483733347 loss:  0.590701335669
Epoch: 2 Step: 1536 loss: 0.460281001093
Val F1 Score:  46.9034574073 loss:  0.616300976276
Epoch: 3 Step: 1584 loss: 0.446875226684
Val F1 Score:  40.2498691784 loss:  0.539125782251
Epoch: 4 Step: 1632 loss: 0.419834913996
Val F1 Score:  39.3014582882 loss:  0.592014420033
Epoch: 5 Step: 1680 loss: 0.408792479895
Val F1 Score:  40.1807549176 loss:  0.601997727156
Epoch: 6 Step: 1728 loss: 0.425682423636
Val F1 Score:  38.0532223966 loss:  0.583338820934
Epoch: 7 Step: 1776 loss: 0.397434794034
Val F1 Score:  39.7836767818 loss:  0.549154347181
Epoch: 8 Step: 1824 loss: 0.394719573669
Val F1 Score:  46.0245539657 loss:  0.57377846241
Epoch: 9 Step: 1872 loss: 0.412809213623
Val F1 Score:  46.2948950647 loss:  0.610173022747
Epoch: 10 Step: 1920 loss: 0.375292135713
Val F1 Score:  43.3982175655 loss:  0.513172185421
Epoch: 11 Step: 1968 loss: 0.398509702335
Val F1 Score:  37.0273214875 loss:  0.